In [ ]:
import cv2
import numpy as np

img = cv2.imread('lena.png')
height, width, channels = img.shape

salt_pepper_img = img.copy()

# Create matrix with random values from 0 to 100 and same height, width as input img
salt_pepper_mat = np.random.randint(0, 101, size=(height, width))

# Apply Salt and Pepper Noise
for row in range (0, height):
    for col in range (0, width):
        if (salt_pepper_mat[row, col] == 0):
            salt_pepper_img[row, col, :] = 0
        if (salt_pepper_mat[row, col] == 100):
            salt_pepper_img[row, col, :] = 255
            
# Set size n for n*n filter 
filter_size = 9

# Create Zero-Padded Salt and Pepper Img 
padded_height = height+filter_size-1
padded_width = width+filter_size-1
padded = np.zeros( (padded_height, padded_width, 3), dtype=np.uint8 )

# Calculate positions where the non-zero data is located & Calculate center of filter
img_height_begin = filter_center_x = int(filter_size/2)
img_height_end = height + int(filter_size/2)
img_width_begin = filter_center_y = int(filter_size/2)
img_width_end = width + int(filter_size/2)

# Put img with salt and pepper into padded matrix
# If row and col to access non-padded matrix, need to minus offset
for row in range ( img_height_begin, img_height_end):
    for col in range ( img_width_begin, img_width_end ):
        padded[row, col] = salt_pepper_img[ (row-img_height_begin), (col-img_width_begin) ]

##### Apply Median Filter #####
median_buffer = salt_pepper_img.copy()
# If row and col to access non-padded max, need to minus offset
for row in range (img_height_begin, img_height_end):
    for col in range (img_width_begin, img_width_end):
        filter_list = []
        for filter_row in range (row-filter_center_x, row+filter_center_x+1):
            for filter_col in range (col-filter_center_y, col+filter_center_y+1):
                filter_list.append( padded[filter_row, filter_col, 1] )
        filter_list.sort()
        median_buffer[row-img_height_begin, col-img_width_begin, :] = filter_list[ int( len(filter_list)/2 ) ]
##### End of Apply Median Filter #####
        
##### Apply Mean Filter #####
mean_buffer = salt_pepper_img.copy()
# If row and col to access non-padded max, need to minus offset
for row in range (img_height_begin, img_height_end):
    for col in range (img_width_begin, img_width_end):
        sum = 0
        for filter_row in range (row-filter_center_x, row+filter_center_x+1):
            for filter_col in range (col-filter_center_y, col+filter_center_y+1):
                sum += padded[filter_row, filter_col, 1] 
        mean = sum / filter_size**2
        mean_buffer[row-img_height_begin, col-img_width_begin, :] = mean
##### End of Apply Mean Filter #####
    
cv2.imshow('input', img)
cv2.imshow('salt and pepper image', salt_pepper_img)
cv2.imshow('median filter', median_buffer)
cv2.imshow('mean filter', mean_buffer)
cv2.imwrite('salt_pepper_img.jpg', salt_pepper_img)
cv2.imwrite('padded_salt_pepper_img.jpg', padded)
cv2.imwrite('9x9 median_filter_result.jpg', median_buffer)
cv2.imwrite('9x9 mean_filter_result.jpg', mean_buffer)

cv2.waitKey(0)
cv2.destroyAllWindows()